In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.neighbors import KNeighborsRegressor
import time
import datetime
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/Shareddrives/Machine Learning - Transformers/

/content/drive/Shareddrives/Machine Learning - Transformers


In [ ]:
def oneHotEncode(df, cols):
  for i in cols:
    x = pd.get_dummies(df.loc[:,i], prefix=i)
    df.drop(i, axis=1, inplace=True)
    df = pd.concat([df,x], axis=1)
  return df

In [ ]:
cols = ['weather', 'wind_direction']

In [ ]:
# final_new_interpolated = pd.read_csv('Data/data_final_transformer.csv')
final_new_interpolated = pd.read_csv('Data/final_new_Interpolated.csv')
final_new_interpolated = oneHotEncode(final_new_interpolated, cols)
final_new_interpolated

,time,station_id,PM25_Concentration,latitude,longitude,temperature,pressure,humidity,wind_speed,weather_0.0,...,wind_direction_0.0,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0
0,2014-05-01 00:00:00,1001,20.0,40.090679,116.173553,20.0,1004.0,56.0,7.92,1,...,0,0,0,0,0,0,1,0,0,0
1,2014-05-01 00:00:00,1002,20.0,40.003950,116.205310,20.0,1004.0,56.0,7.92,1,...,0,0,0,0,0,0,1,0,0,0
2,2014-05-01 00:00:00,1003,20.0,39.914409,116.184239,20.0,1004.0,56.0,7.92,1,...,0,0,0,0,0,0,1,0,0,0
3,2014-05-01 00:00:00,1004,20.0,39.815128,116.171150,20.0,1004.0,56.0,7.92,1,...,0,0,0,0,0,0,1,0,0,0
4,2014-05-01 00:00:00,1005,19.0,39.742767,116.136045,19.0,1002.0,61.0,11.16,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315355,2015-04-30 23:00:00,1032,24.4,40.499000,116.911000,24.4,1004.0,47.0,2.70,0,...,0,0,0,0,0,0,0,0,1,0
315356,2015-04-30 23:00:00,1033,24.4,40.100000,117.120000,24.4,1004.0,47.0,2.70,0,...,0,0,0,0,0,0,0,0,1,0
315357,2015-04-30 23:00:00,1034,24.4,39.712000,116.783000,24.4,1004.0,47.0,2.70,0,...,0,0,0,0,0,0,0,0,1,0
315358,2015-04-30 23:00:00,1035,24.4,39.520000,116.300000,24.4,1004.0,47.0,2.70,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
final_mm = pd.read_csv('Data/final_mm.csv')
final_mm = oneHotEncode(final_mm, cols)
# final_mm
final_std = pd.read_csv('Data/final_std.csv')
final_std = oneHotEncode(final_std, cols)
# final_std

In [ ]:
final_mm[final_mm["station_id"]==1035].head()

,station_id,time,PM25_Concentration,latitude,longitude,district_id,temperature,pressure,humidity,wind_speed,...,wind_direction_0.0,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0
263193,1035,2014-05-01 00:00:00,112.0,39.52,116.3,108,0.614035,0.66,0.62,0.096296,...,0,0,0,0,0,0,1,0,0,0
263194,1035,2014-05-01 01:00:00,123.0,39.52,116.3,108,0.614035,0.66,0.67,0.066667,...,0,0,0,0,0,0,1,0,0,0
263195,1035,2014-05-01 02:00:00,143.0,39.52,116.3,108,0.614035,0.65,0.71,0.096296,...,0,0,0,0,0,0,1,0,0,0
263196,1035,2014-05-01 03:00:00,138.0,39.52,116.3,108,0.614035,0.66,0.72,0.096296,...,0,0,0,0,0,0,1,0,0,0
263197,1035,2014-05-01 04:00:00,145.0,39.52,116.3,108,0.596491,0.65,0.77,0.051852,...,0,1,0,0,0,0,0,0,0,0


# LOOKING AT THE DATA

In [ ]:
final_new_interpolated.head()

,station_id,time,PM25_Concentration,latitude,longitude,district_id,temperature,pressure,humidity,wind_speed,...,wind_direction_0.0,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0
0,1001,2014-05-01 00:00:00,138.0,40.090679,116.173553,101,20.0,1004.0,56.0,7.92,...,0,0,0,0,0,0,1,0,0,0
1,1001,2014-05-01 01:00:00,124.0,40.090679,116.173553,101,18.0,1004.0,64.0,7.56,...,0,0,0,0,0,0,1,0,0,0
2,1001,2014-05-01 02:00:00,127.0,40.090679,116.173553,101,18.0,1004.0,70.0,5.76,...,0,0,0,0,0,0,1,0,0,0
3,1001,2014-05-01 03:00:00,129.0,40.090679,116.173553,101,17.0,1004.0,74.0,6.12,...,0,0,0,0,0,0,1,0,0,0
4,1001,2014-05-01 04:00:00,119.0,40.090679,116.173553,101,17.0,1004.0,75.0,4.68,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
final_mm.head()

,station_id,time,PM25_Concentration,latitude,longitude,district_id,temperature,pressure,humidity,wind_speed,...,wind_direction_0.0,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0
0,1001,2014-05-01 00:00:00,138.0,40.090679,116.173553,101,0.631579,0.66,0.56,0.162963,...,0,0,0,0,0,0,1,0,0,0
1,1001,2014-05-01 01:00:00,124.0,40.090679,116.173553,101,0.596491,0.66,0.64,0.155556,...,0,0,0,0,0,0,1,0,0,0
2,1001,2014-05-01 02:00:00,127.0,40.090679,116.173553,101,0.596491,0.66,0.70,0.118519,...,0,0,0,0,0,0,1,0,0,0
3,1001,2014-05-01 03:00:00,129.0,40.090679,116.173553,101,0.578947,0.66,0.74,0.125926,...,0,0,0,0,0,0,1,0,0,0
4,1001,2014-05-01 04:00:00,119.0,40.090679,116.173553,101,0.578947,0.66,0.75,0.096296,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
final_std.head()

,station_id,time,PM25_Concentration,latitude,longitude,district_id,temperature,pressure,humidity,wind_speed,...,wind_direction_0.0,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0
0,1001,2014-05-01 00:00:00,138.0,40.090679,116.173553,101,0.458954,0.071874,0.241174,0.147845,...,0,0,0,0,0,0,1,0,0,0
1,1001,2014-05-01 01:00:00,124.0,40.090679,116.173553,101,0.281086,0.071874,0.578746,0.072943,...,0,0,0,0,0,0,1,0,0,0
2,1001,2014-05-01 02:00:00,127.0,40.090679,116.173553,101,0.281086,0.071874,0.831925,-0.301567,...,0,0,0,0,0,0,1,0,0,0
3,1001,2014-05-01 03:00:00,129.0,40.090679,116.173553,101,0.192152,0.071874,1.000711,-0.226665,...,0,0,0,0,0,0,1,0,0,0
4,1001,2014-05-01 04:00:00,119.0,40.090679,116.173553,101,0.192152,0.071874,1.042908,-0.526273,...,0,1,0,0,0,0,0,0,0,0


# Creating months attribute

In [ ]:
months = pd.Series( [ ( datetime.datetime.strptime(time,'%Y-%m-%d %H:%M:%S').month ) for time in final_new_interpolated['time'] ], copy = True )
months

0         5
1         5
2         5
3         5
4         5
         ..
315355    4
315356    4
315357    4
315358    4
315359    4
Length: 315360, dtype: int64

# Randomly taking 24 stations as Train and 12 as test

In [ ]:
final_new_interpolated.shape

(315360, 35)

In [ ]:
final_months = []
final_months_std = []
final_months_mm = []
for month in range(1, 13):
  final_months.append(final_new_interpolated[months == month])
  final_months_mm.append(final_mm[months == month])
  final_months_std.append(final_std[months == month])
final_months[7]

,time,station_id,PM25_Concentration,latitude,longitude,temperature,pressure,humidity,wind_speed,weather_0.0,...,wind_direction_0.0,wind_direction_1.0,wind_direction_2.0,wind_direction_3.0,wind_direction_4.0,wind_direction_9.0,wind_direction_13.0,wind_direction_14.0,wind_direction_23.0,wind_direction_24.0
79488,2014-08-01 00:00:00,1001,26.0,40.090679,116.173553,26.0,1000.718922,83.0,6.606830,1,...,0,0,0,0,0,0,0,0,1,0
79489,2014-08-01 00:00:00,1002,26.0,40.003950,116.205310,26.0,1002.793627,83.0,6.128480,1,...,0,0,0,0,0,0,0,0,1,0
79490,2014-08-01 00:00:00,1003,26.0,39.914409,116.184239,26.0,1006.338016,85.0,4.167255,1,...,0,0,0,0,1,0,0,0,0,0
79491,2014-08-01 00:00:00,1004,27.0,39.815128,116.171150,27.0,1006.035506,77.0,4.096993,1,...,0,0,0,0,0,0,0,0,1,0
79492,2014-08-01 00:00:00,1005,25.0,39.742767,116.136045,25.0,1001.000000,82.0,6.480000,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106267,2014-08-31 23:00:00,1032,21.0,40.499000,116.911000,21.0,1006.000000,89.0,1.080000,0,...,0,0,0,0,0,0,0,1,0,0
106268,2014-08-31 23:00:00,1033,20.0,40.100000,117.120000,20.0,1010.000000,96.0,1.800000,0,...,0,1,0,0,0,0,0,0,0,0
106269,2014-08-31 23:00:00,1034,22.0,39.712000,116.783000,22.0,1008.440866,94.0,2.024809,0,...,0,0,0,0,1,0,0,0,0,0
106270,2014-08-31 23:00:00,1035,21.0,39.520000,116.300000,21.0,1009.000000,93.0,2.160000,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
month = 7

In [ ]:
stations = np.unique(final_months[month]['station_id'])

In [ ]:
splitter = KFold(n_splits=3, shuffle=True, random_state=1234)

for fold, (train, test) in enumerate(splitter.split(stations)):
    train_stations = stations[train]
    test_stations = stations[test]
    train_data = final_months[month][final_months[month].station_id.isin(train_stations)]
    test_data = final_months[month][final_months[month].station_id.isin(test_stations)]

    train_data_mm = final_months_mm[month][final_months_mm[month].station_id.isin(train_stations)]
    test_data_mm = final_months_mm[month][final_months_mm[month].station_id.isin(test_stations)]

    train_data_std = final_months_std[month][final_months_std[month].station_id.isin(train_stations)]
    test_data_std = final_months_std[month][final_months_std[month].station_id.isin(test_stations)]
    
    # train_data.to_csv('Data/data_final_transformers_'+str(fold)+'_train.csv', index=None)
    # test_data.to_csv('Data/data_final_transformers_'+str(fold)+'_test.csv', index=None)
    train_data.to_csv('Data/interpolated/fold_'+str(fold)+'_train.csv', index=None)
    test_data.to_csv('Data/interpolated/fold_'+str(fold)+'_test.csv', index=None)

    train_data_mm.to_csv('Data/interpolated/fold_'+str(fold)+'_train_mm.csv', index=None)
    test_data_mm.to_csv('Data/interpolated/fold_'+str(fold)+'_test_mm.csv', index=None)

    train_data_std.to_csv('Data/interpolated/fold_'+str(fold)+'_train_std.csv', index=None)
    test_data_std.to_csv('Data/interpolated/fold_'+str(fold)+'_test_std.csv', index=None)